In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from matplotlib import cm
from sklearn.metrics import silhouette_samples
pd.set_option('mode.chained_assignment',  None)

d1 = pd.read_csv('Indicators.csv')
d2 = pd.read_csv('Country.csv')

data = d1.copy()
df_2 = d2.copy()

# 처음 dataframe만들 때만 사용
# df : indicators.csv
# indicator : 사용할 indicator
# year : 선택할 year
def make_df(df,indicator,year):
    df_1 = df[df['Year'] == year]
    df_1 = df_1[df_1['IndicatorName'] == indicator]
    df_1 = df_1[['CountryCode','Value']]
    return df_1


# data : indicators.csv
# df : merge하기전 dataframe
# indicator : 새로 merge하고싶은 indicator
# year : mearge 하기 위한 year 선택
# df_2 : 새로 merge하기위해 만들어진 dataframe
def data_merge(df,indicator,year):
    df_2 = data[data['Year'] == year]
    df_2 = df_2[df_2['IndicatorName'] == indicator]
    df_2 = df_2[['CountryCode','Value']]
    df_merge = pd.merge(df, df_2, on='CountryCode', how='outer')
    #df_merge.columns = ['CountryCode', indicator+'_value',indicator2+'_value']
    #df_merge['Year'] = year
    return df_merge

#'Birth rate, crude (per 1,000 people)'
# Using indicators list
indi_list = ['Life expectancy at birth, total (years)','Age dependency ratio, old (% of working-age population)','Birth rate, crude (per 1,000 people)']
# 1st DataFrame
#df1 = make_df(data,indi_list[0],2000)
# Merge 1 indicator
#merge_1 = data_merge(df1,indi_list[1],2000)
# Merge 1 indicator 
#merge_2 = data_merge(merge_1,indi_list[2],2000)

#merge_2.columns = ['CountryCode',indi_list[0],indi_list[1],indi_list[2]]

#merge_last = merge_2.copy()

# Check the Null value
def check_isnan_list(df,col):
    null_list = []
    for i in range(len(df)):
        if math.isnan(df[col][i]):
            null_list.append(i)
        else:
            continue
            
    return null_list



# Check the Null value
def check_isnan_dict(df,col):
    null_list = []
    for i in range(len(df)):
        if math.isnan(df[col][i]):
            null_list.append(i)
        else:
            continue
            
    null_dict = dict()
    for country in df['CountryCode'][null_list]:
        for index in range(len(df_2)):
            if df_2['CountryCode'][index] == country:
                region = df_2['Region'][index]
                null_dict[country] = region
    return null_dict

# df1 : Indicator.csv , df2 : Country.csv
def Check_country_IncomeGroup(df1,df2,group,indicator,year):
    country_group = df2[df2['IncomeGroup'] == group]['CountryCode']
    data_group = df1[df1['CountryCode'].isin(country_group)]
    return data_group[(data_group['IndicatorName'] == indicator) & (data_group['Year'] == year)]

# df1 : Indicator.csv , df2 : Country.csv
def Check_country_region(df1,df2,region,indicator,year):
    country_region = df2[df2['Region'] == region]['CountryCode']
    data_region = df1[df1['CountryCode'].isin(country_region)]
    return data_region[(data_region['IndicatorName'] == indicator) & (data_region['Year'] == year)]

# df1 : Indicator.csv , df2 : Country.csv
def Check_CurrencyUnit(df1,df2,currencyunit,indicator,year):
    unit = df2[df2['CurrencyUnit'] == currencyunit]['CountryCode']
    data_unit = df1[df1['CountryCode'].isin(unit)]
    return data_unit[(data_unit['IndicatorName'] == indicator) & (data_unit['Year'] == year)]

# Check the Null value
# df : merge가 끝난 dataframe
# col : null값을 채우기 위한 indicator value
# year : 해당 데이터셋의 year
# method : null값을 채우기 위한 기준
def change_null_value(df,col,year,method):
    null_list = []
    for i in range(len(df)):
        if math.isnan(df[col][i]):
            null_list.append(i)
        else:
            continue
            
    null_dict = dict()
    if method =='Region':
        for country in df['CountryCode'][null_list]:
            for index in range(len(df_2)):
                if df_2['CountryCode'][index] == country:
                    region = df_2[method][index]
                    null_dict[country] = region
        keys = list(null_dict.keys())        
        for key,null_value in zip(keys,null_list):
            new_value_data = Check_country_region(data,df_2,null_dict[key],col,year)
            new_value = np.average(new_value_data['Value'])
            #dff = dff.T
            #dff.at[col,null_value] = new_value
            #dff = dff.T
            df[col][null_value] = new_value
        return df
    
    elif method =='IncomeGroup':
        for country in df['CountryCode'][null_list]:
            for index in range(len(df_2)):
                if df_2['CountryCode'][index] == country:
                    group = df_2[method][index]
                    null_dict[country] = group
        keys = list(null_dict.keys())        
        for key,null_value in zip(keys,null_list):
            new_value_data = Check_country_IncomeGroup(data,df_2,null_dict[key],col,year)
            new_value = np.average(new_value_data['Value'])
            df[col][null_value] = new_value       
        return df
    
    elif method == 'CurrencyUnit':
        for country in df['CountryCode'][null_list]:
            for index in range(len(df_2)):
                if df_2['CountryCode'][index] == country:
                    unit = df_2[method][index]
                    null_dict[country] = unit
        keys = list(null_dict.keys())        
        for key,null_value in zip(keys,null_list):
            new_value_data = Check_CurrencyUnit(data,df_2,null_dict[key],col,year)
            new_value = np.average(new_value_data['Value'])
            df[col][null_value] = new_value       
        return df
    
def merge_null_change(data,year,method):
    # Using indicators list
    indi_list = ['Life expectancy at birth, total (years)','Age dependency ratio, old (% of working-age population)','Birth rate, crude (per 1,000 people)']
    # 1st DataFrame
    df1 = make_df(data,indi_list[0],year)
    # Merge 1 indicator
    merge_1 = data_merge(df1,indi_list[1],year)
    # Merge 1 indicator 
    merge_2 = data_merge(merge_1,indi_list[2],year)
    merge_2.columns = ['CountryCode',indi_list[0],indi_list[1],indi_list[2]]
    merge_last = merge_2.copy()

    df_method= merge_last.copy()
    for i in range(3):
        df_method = change_null_value(df_method,indi_list[i],year,method)
    return df_method

## DBSCAN ##
    # Make the list for using compare the result for various parameter combinations
def Show_DBSCAN(df):
    eps = [0.001,0.005,0.01,0.02,0.05,0.1,0.2,0.5]
    min_samples= [3,5,10,15,20,30,50,100]
    for i in eps:
        for j in min_samples:
            dbscan = DBSCAN(eps = i, min_samples = j).fit(df)
            labels_dbscan = dbscan.labels_
            dbscan_cluster_num = labels_dbscan.max() +2
            plt.figure(figsize=(9,9))
            plt.scatter(df['P1'],df['P2'],c=labels_dbscan)
            plt.title("DBSCAN Scatter - eps: {}, min_samples: {}, cluster_num: {}".format(i,j,dbscan_cluster_num))
            plt.show()
            
## KMeans ##

    # Make the list for using compare the result for various parameter combinations
def Show_KMeans(df):
    n_clusters = [2,3,4,5,6]
    max_iter= [50,100,200,300]
    for i in n_clusters: 
        for j in max_iter: 
                # Make the Kmeans instance and fit
            Kmeans = KMeans(n_clusters=i, max_iter = j).fit(df)
                # Make to list for kmeans labels value
            labels_kmeans = Kmeans.labels_
                # Count the cluster num
            kmeans_cluster_num = labels_kmeans.max() + 1
        
                ## Plot the Scatter each of parameters ##
            plt.figure(figsize =(9, 9)) 
            plt.scatter(df['P1'], df['P2'], c = labels_kmeans)   
                # Building the title
            plt.title("KMeans Scatter - n_clusters: {}, max_iter: {}, cluster_num: {}".format(i,j,kmeans_cluster_num))
            plt.show() 
            
## GMM ##

    # Make the list for using compare the result for various parameter combinations
def Show_GMM(df):
    n_components = [2,3,4,5,6]
    max_iter= [50,100,200,300]
    for i in n_components: 
        for j in max_iter:
                # Make the GMM instance and fit
            gmm = GMM(n_components= i , max_iter = j).fit(df)
                # Make to list for GMM labels value
            labels_gmm = gmm.predict(df)
                # Count the cluster num
            gmm_cluster_num = labels_gmm.max() + 1
        
                ## Plot the Scatter each of parameters ##
            plt.figure(figsize =(9, 9)) 
            plt.scatter(df['P1'], df['P2'], c = labels_gmm)   
                # Building the title
            plt.title("GMM Scatter - n_components: {},max_iter: {}, cluster_num: {}".format(i,j,gmm_cluster_num))
            plt.show()             


def plotSilhouette(data, cluster_label):
    cluster_labels = np.unique(cluster_label)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_samples(data, cluster_label, metric='euclidean')
    y_ax_lower, y_ax_upper = 0,0
    yticks = []
    
    for i, c in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[cluster_label ==c]
        c_silhouette_vals.sort()
        y_ax_upper += len(c_silhouette_vals)
        color = cm.jet(i/n_clusters)
        
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0,
                    edgecolor = 'none', color = color)
        yticks.append((y_ax_lower + y_ax_upper)/2)
        y_ax_lower += len(c_silhouette_vals)
    silhoutte_avg = np.mean(silhouette_vals)
    #plt.figure(figsize=(9,9))
    plt.axvline(silhoutte_avg, color='red', linestyle='--')
    plt.yticks(yticks, cluster_labels+1)
    plt.ylabel('cluster')
    plt.xlabel('silhoueete value')
  #  plt.title(" Silhouette - eps: {}, min_samples: {}, cluster_num: {}".format(i,j,dbscan_cluster_num))
    plt.show()

data_1960_region = merge_null_change(data,1960,'Region')
data_1980_region = merge_null_change(data,1980,'Region')
data_2000_region = merge_null_change(data,2000,'Region')
data_2010_region = merge_null_change(data,2010,'Region')

data_1960_group = merge_null_change(data,1960,'IncomeGroup')
data_1980_group = merge_null_change(data,1980,'IncomeGroup')
data_2000_group = merge_null_change(data,2000,'IncomeGroup')
data_2010_group = merge_null_change(data,2010,'IncomeGroup')

data_1960_unit = merge_null_change(data,1960,'CurrencyUnit')
data_1980_unit = merge_null_change(data,1980,'CurrencyUnit')
data_2000_unit = merge_null_change(data,2000,'CurrencyUnit')
data_2010_unit = merge_null_change(data,2010,'CurrencyUnit')


encoder = LabelEncoder()

data_region = pd.concat([data_1960_region,data_1980_region,data_2000_region,data_2010_region])
data_region['CountryCode'] = encoder.fit_transform(data_region['CountryCode'])

data_group = pd.concat([data_1960_group,data_1980_group,data_2000_group,data_2010_group])
data_group['CountryCode'] = encoder.fit_transform(data_group['CountryCode'])

data_unit = pd.concat([data_1960_unit,data_1980_unit,data_2000_unit,data_2010_unit])
data_unit = data_unit.dropna(axis=0)
data_unit['CountryCode'] = encoder.fit_transform(data_unit['CountryCode'])

## Preprocessing the data ##

# Make the MinMaxScaler instance and StandardScaler instance
# Compare the result by using two Scaler
scaler_mm = MinMaxScaler(feature_range=(0, 1))
scaler_std = StandardScaler()

# Fit the DataFrame for each of Scaler
df_scaled_mm_region = scaler_mm.fit_transform(data_region)
df_scaled_std_region = scaler_std.fit_transform(data_region)

# Fit the DataFrame for each of Scaler
df_scaled_mm_group = scaler_mm.fit_transform(data_group)
df_scaled_std_group = scaler_std.fit_transform(data_group)

# Fit the DataFrame for each of Scaler
df_scaled_mm_unit = scaler_mm.fit_transform(data_unit)
df_scaled_std_unit = scaler_std.fit_transform(data_unit)


# Change the normalized and Make the DataFrame for each of Scaler

df_normalized_mm_region = normalize(df_scaled_mm_region)
df_normalized_mm_region = pd.DataFrame(df_normalized_mm_region)

df_normalized_std_region = normalize(df_scaled_std_region)
df_normalized_std_region = pd.DataFrame(df_normalized_std_region)

# Change the normalized and Make the DataFrame for each of Scaler

df_normalized_mm_group = normalize(df_scaled_mm_group)
df_normalized_mm_group = pd.DataFrame(df_normalized_mm_group)

df_normalized_std_group = normalize(df_scaled_std_group)
df_normalized_std_group = pd.DataFrame(df_normalized_std_group)


# Change the normalized and Make the DataFrame for each of Scaler

df_normalized_mm_unit = normalize(df_scaled_mm_unit)
df_normalized_mm_unit = pd.DataFrame(df_normalized_mm_unit)

df_normalized_std_unit = normalize(df_scaled_std_unit)
df_normalized_std_unit = pd.DataFrame(df_normalized_std_unit)


## Reducing the dimensionality of the data by using the PCA ##

# Make the PCA instance
pca = PCA(n_components = 2)

# Fit the data and Make the DataFrame for each of Scaler

df_principal_mm_region = pca.fit_transform(df_normalized_mm_region)
df_principal_mm_region = pd.DataFrame(df_principal_mm_region)
df_principal_mm_region.columns = ['P1','P2']

df_principal_std_region = pca.fit_transform(df_normalized_std_region)
df_principal_std_region = pd.DataFrame(df_principal_std_region)
df_principal_std_region.columns = ['P1','P2']


# Fit the data and Make the DataFrame for each of Scaler

df_principal_mm_group = pca.fit_transform(df_normalized_mm_group)
df_principal_mm_group = pd.DataFrame(df_principal_mm_group)
df_principal_mm_group.columns = ['P1','P2']

df_principal_std_group = pca.fit_transform(df_normalized_std_group)
df_principal_std_group = pd.DataFrame(df_principal_std_group)
df_principal_std_group.columns = ['P1','P2']


# Fit the data and Make the DataFrame for each of Scaler

df_principal_mm_unit = pca.fit_transform(df_normalized_mm_unit)
df_principal_mm_unit = pd.DataFrame(df_principal_mm_unit)
df_principal_mm_unit.columns = ['P1','P2']

df_principal_std_unit = pca.fit_transform(df_normalized_std_unit)
df_principal_std_unit = pd.DataFrame(df_principal_std_unit)
df_principal_std_unit.columns = ['P1','P2']


Show_DBSCAN(df_principal_mm_region)
Show_KMeans(df_principal_mm_region)
Show_GMM(df_principal_mm_region)

Show_DBSCAN(df_principal_std_region)
Show_KMeans(df_principal_std_region)
Show_GMM(df_principal_std_region)

Show_DBSCAN(df_principal_mm_group)
Show_KMeans(df_principal_mm_group)
Show_GMM(df_principal_mm_group)

Show_DBSCAN(df_principal_std_group)
Show_KMeans(df_principal_std_group)
Show_GMM(df_principal_std_group)

Show_DBSCAN(df_principal_mm_unit)
Show_KMeans(df_principal_mm_unit)
Show_GMM(df_principal_mm_unit)

Show_DBSCAN(df_principal_std_unit)
Show_KMeans(df_principal_std_unit)
Show_GMM(df_principal_std_unit)


# 실루엣 계수는 -1 에서 1사이의 값을 가지며, 
# 1로 가까워 질수록 근처의 군집과 더 멀리 떨어져 있다는 것이고, 
# 0에 가까울 수록 근처의 군집과 가까워 진다는 것이다.
# - 값은 아예 다른 군집에 데이터 포인트가 할당 됐음을 뜻한다.

# 눈으로 먼저 몇개를 고른다음에 고른 것 중에서 실루엣 계수를 그린다음에 마무리 해야할 것 같아요.
# purity는 정답 label이 없기에 쓸 수 없는것 같습니다.

#dbscan = DBSCAN(eps = 0.05, min_samples = 10).fit(df_principal_mm_region)
#labels_dbscan = dbscan.labels_
#sil = dbscan.fit_predict(df_principal_mm_region)

#plotSilhouette(df_principal_mm_region, sil)